## Plot 1-sigma ellipse

In [ ]:
# adopt from Fisher.py, updated 
from numpy import sqrt, array, pi, arctan, where, less, greater, logical_and 
def setnd(nd=2):
    """
    Set sigma for confidence contours
    nd = # dimensions (# parameters)
    Normally this should be 2
    unless there is a *total* degeneracy, unless it should be 1
    """
    global nsig, nsig2, nsig1, nsig0
    if nd == 2:
        nsig = sqrt(array([6.17, 2.3, 0]))
    elif nd == 1:
        nsig = array([2, 1, 0])
    nsig2, nsig1, nsig0 = nsig
    return nsig

def atanxy(x, y, degrees=0):
    """ANGLE CCW FROM x-axis"""
    theta = arctan(divsafe(y, x, inf=1e30, nan=0))
    theta = where(less(x, 0), theta + pi, theta)
    theta = where(logical_and(greater(x, 0), less(y, 0)), theta + 2*pi, theta)
    if degrees:
        theta = theta * 180. / pi
    return theta

def setell(dx, dy, p, nD=2):
    """
    Return ellipse parameters A, B, ang[degrees] given dx, dy, p
    dx, dy = 1-sigma uncertainties in x, y
    p = correlation coefficient (0 = independent, 1 = completely correlated)
    Eqs. 2,3,4 from Fisher Quick Start Guide"""
    # Ellipse formulae
    # http://www.scribd.com/doc/2341150/Lecture-5-Bivariate-ND-Error-Ellipses
    dxy = p * dx * dy
    
    de1 = (dx**2 + dy**2) / 2
    de2 = (dx**2 - dy**2)**2 / 4
    de3 = sqrt(de2 + dxy**2)
    A = sqrt(de1 + de3)
    B = sqrt(de1 - de3)
    if nD == 1:
        A = A * sqrt(2)
        B = B * sqrt(2)
    
    sin2ang = 2 * dxy # / something
    cos2ang = dx**2 - dy**2 # / something
    ang = atanxy(cos2ang, sin2ang) / 2
    return A, B, ang

from matplotlib.patches import Ellipse as Ellipse
def ellpatch1(x, y, w, h, ang, fc, ec, alpha=1, zorder=1, fill=1):
    patch = Ellipse((x,y), w, h, ang*180/pi, fill=fill)
    patch.set_fc(fc)
    patch.set_ec(ec)
    patch.set_alpha(alpha)
    patch.set_zorder(zorder)
    return patch

def plotell1p(xo, yo, dx, dy, p, color='b', fillcolor=None, alpha=1, zorder=1, lw=1, fill=False, nD=2):
    """
    Plot 1-sigma ellipse given dx, dy, p
    dx, dy = 1-sigma uncertainties in x, y
    p = correlation coefficient (0 = independent, 1 = completely correlated)
    """
    setnd(nD)
    if fill and fillcolor==None:
        fillcolor = color
    A, B, ang = setell(dx, dy, p)
    patch = ellpatch1(xo, yo, 2*nsig1*A, 2*nsig1*B, ang, 
                      fillcolor, color, alpha, zorder, fill)
    patch.set_lw(lw)
    gca().add_patch(patch)